In [ ]:
import os
os.chdir(r'C:\Users\maint4\My Drive\MCH Sale forecasting\Works\data_SO_SI_weekly_by_channel')

In [ ]:
# pipeline
from imblearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBRegressor

In [ ]:
# data manipulation
import numpy as np
import pandas as pd
from math import sqrt
import isoweek

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# time
import datetime as dt
import time as time
import calendar
import holidays
# pipeline


# models
from sklearn.ensemble import RandomForestRegressor

# optimization and hyperparameters
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# model evaluation

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error,mean_squared_error

from statsmodels.tsa.seasonal import seasonal_decompose

# saving models
from joblib import dump

# models
from sklearn.ensemble import RandomForestRegressor

# optimization and hyperparameters
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# model evaluation
from sklearn.metrics import mean_squared_error


import os
import glob
file_path = r'C:\Users\maint4\My Drive\MCH Sale forecasting\Works\data_SO_SI_weekly_by_channel'
#list all the files from the directory
file_list = os.listdir(file_path)

csv_files = glob.glob('*.{}'.format('csv'))
#combine all files in the list
df_20_23 = pd.read_csv('bq-results-20231206-110541-1701860782693.csv')
inventory_df = pd.read_csv('GT0_Inventory_weekly.csv')
#pd.concat([pd.read_csv(f) for f in file_list])
Tet_Holiday = pd.read_excel('Master_Date.xlsx', sheet_name= 'Holiday_count')
Tet = pd.read_excel('Master_Date.xlsx', sheet_name= 'Tet_Holiday')

In [ ]:
df_20_23.head()

1. Data processing

In [ ]:
SI_NA = df_20_23[df_20_23["STANDARD_SKU_DESC"].isnull() & df_20_23["STANDARD_SKU_DESC_1"].notnull()]
SO_NA = df_20_23[df_20_23["STANDARD_SKU_DESC"].notnull() & df_20_23["STANDARD_SKU_DESC_1"].isnull()]
SI_SO = df_20_23[df_20_23["STANDARD_SKU_DESC"].notnull() & df_20_23["STANDARD_SKU_DESC_1"].notnull()]


#SO preparation
SO_NA['WEEK_NUM'] = SO_NA['WEEK_NUM_SI']

print(SO_NA)

SO_NA_rm_col =  SO_NA.drop(['WEEK_NUM_SI','WEEK_NUM_SO','YEAR_1','CITY_1','REGION_1','STANDARD_SKU_DESC_1'
                            , 'SUB_DIVISION_DESC_1','CATEGORY_DESC_1', 'SUB_CATEGORY_DESC_1','CHANNEL_CODE_1'], axis = 1)

SO_NA_rm_col['WEEKLY_SO_ORDER'] = SO_NA_rm_col['WEEKLY_SO_ORDER'].fillna(0)


In [ ]:
#SI preparation

SI_NA['WEEK_NUM'] = SI_NA['WEEK_NUM_SO']
SI_NA['STANDARD_SKU_DESC'] = SI_NA['STANDARD_SKU_DESC_1']
SI_NA['YEAR'] = SI_NA['YEAR_1']
SI_NA['CHANNEL_CODE'] = SI_NA['CHANNEL_CODE_1']
SI_NA['CITY'] = SI_NA['CITY_1']
SI_NA['REGION'] = SI_NA['REGION_1']
SI_NA['SUB_DIVISION_DESC'] = SI_NA['SUB_DIVISION_DESC_1']
SI_NA['CATEGORY_DESC'] = SI_NA['CATEGORY_DESC_1']
SI_NA['SUB_CATEGORY_DESC'] = SI_NA['SUB_CATEGORY_DESC_1']

SI_NA['WEEKLY_SI_ORDER'] = SI_NA['WEEKLY_SI_ORDER'].fillna(0)


SI_NA_rm_col =  SI_NA.drop(['WEEK_NUM_SI','WEEK_NUM_SO','YEAR_1','CITY_1','REGION_1','STANDARD_SKU_DESC_1'
                            , 'SUB_DIVISION_DESC_1','CATEGORY_DESC_1', 'SUB_CATEGORY_DESC_1','CHANNEL_CODE_1'], axis = 1)


In [ ]:
# SI_SO 
SI_SO['WEEK_NUM'] = SI_SO['WEEK_NUM_SI']
SI_SO_rm_col =  SI_SO.drop(['WEEK_NUM_SI','WEEK_NUM_SO','YEAR_1','CITY_1','REGION_1','STANDARD_SKU_DESC_1'
                            , 'SUB_DIVISION_DESC_1','CATEGORY_DESC_1', 'SUB_CATEGORY_DESC_1','CHANNEL_CODE_1'], axis = 1)

df = pd.concat([SI_SO_rm_col,SO_NA_rm_col,SI_NA_rm_col ], ignore_index=True)



In [ ]:
df = df.drop(['CHANNEL_CODE'], axis = 1)

In [ ]:
STANDARD_SKU = df[['STANDARD_SKU_DESC','SUB_DIVISION_DESC','CATEGORY_DESC','SUB_CATEGORY_DESC']].drop_duplicates()
CITY_REGION = df[['CITY', 'REGION']].drop_duplicates()
n_YEAR = df['YEAR'].unique()

In [ ]:
WEEK_NUM = []
YEAR = []
for i in n_YEAR:
    for j in range(1,isoweek.Week.last_week_of_year(int(i)).week+1):
        YEAR.append(i)
        WEEK_NUM.append(j)

print(YEAR, WEEK_NUM)

list_of_tuples = list(zip(YEAR, WEEK_NUM))

WEEK_NUM_YEAR = pd.DataFrame(list_of_tuples,  columns= ['YEAR','WEEK_NUM'])

In [ ]:
data = pd.merge(STANDARD_SKU, CITY_REGION, how = 'cross').merge(WEEK_NUM_YEAR, how = 'cross')

In [ ]:
dataset = pd.merge(data,df, how = 'left', on = ['STANDARD_SKU_DESC', 'REGION', 'CITY', 'WEEK_NUM', 'YEAR','SUB_DIVISION_DESC','CATEGORY_DESC','SUB_CATEGORY_DESC'])

In [ ]:
dataset_full = pd.merge(dataset, inventory_df, how = 'left', on = ['STANDARD_SKU_DESC', 'REGION', 'CITY', 'WEEK_NUM', 'YEAR','SUB_DIVISION_DESC','CATEGORY_DESC','SUB_CATEGORY_DESC'])

In [ ]:
dataset_full_21_23 = dataset_full[dataset_full['YEAR'] != 2020].drop(['CHANNEL_CODE'], axis = 1)

dataset_full_21_23['WEEKLY_SI_ORDER'] = dataset_full_21_23['WEEKLY_SI_ORDER'].fillna(0)
dataset_full_21_23['WEEKLY_SO_ORDER'] = dataset_full_21_23['WEEKLY_SO_ORDER'].fillna(0)
dataset_full_21_23['LAST_WEEK_INVENTORY'] = dataset_full_21_23['LAST_WEEK_INVENTORY'].fillna(0)

In [ ]:
#get weeknumber
year_week = dataset_full_21_23['YEAR'].astype(int).astype(str) +'-' + dataset_full_21_23['WEEK_NUM'].astype(int).astype(str)
dt.datetime.strptime('2020-49' + '-1', "%Y-%W-%w")
period_tag = []

for i in year_week:
    d = dt.datetime.strptime(i + '-1', "%Y-%W-%w")
    period_tag.append(d)

dataset_full_21_23['PERIOD_TAG'] = period_tag


In [ ]:
biggest_SKU = dataset_full.copy()
biggest_SKU = biggest_SKU.groupby(['STANDARD_SKU_DESC'])[['WEEKLY_SI_ORDER']].sum()
biggest_SKU.sort_values(by = ['WEEKLY_SI_ORDER'], ascending = False, inplace = True)
biggest_SKU.reset_index(inplace=True)

print(biggest_SKU)

In [ ]:
df2 = dataset_full_21_23.groupby(['STANDARD_SKU_DESC','PERIOD_TAG', 'CITY', 'REGION','YEAR', 'WEEK_NUM', 'SUB_DIVISION_DESC', 
                  'CATEGORY_DESC', 'SUB_CATEGORY_DESC' ]).agg({'WEEKLY_SI_ORDER' : 'sum', 'WEEKLY_SO_ORDER' : 'sum','LAST_WEEK_INVENTORY' : 'sum' }).reset_index()


In [ ]:
df2['PROVINCE_REGION'] = df2['CITY']+'-'+df2['REGION']
df2 = df2.drop(['CITY', 'REGION'], axis = 1)
df2['MONTH'] = df2['PERIOD_TAG'].dt.month

In [ ]:
#add holiday data
df2 = df2.merge(Tet_Holiday, how = 'left', left_on=['WEEK_NUM', 'YEAR'], right_on= ['Iso Weeknum','Iso Year'])
df2=df2.drop(['Iso Year','Iso Weeknum'], axis= 1) #,'Holiday_count_x','Week_Year_x',	'Iso Year_y',	'Iso Weeknum_y',	'Holiday_count_y',	'Week_Year_y',	'Iso Year',	'Iso Weeknum',	'Holiday_count',	'Week_Year'
df2.rename(columns = {'Holiday_count' : 'HOLIDAY_COUNT', 'N_Working_day' : 'WORKING_DAY_COUNT'}, inplace = True)
df2.head()

In [ ]:
# separated variable for Tet Holiday occassion
df2 = df2.merge(Tet, how = 'left', left_on=['WEEK_NUM', 'YEAR'], right_on= ['Iso Weeknum','Iso Year'])
df2=df2.drop(['Iso Year','Iso Weeknum'], axis= 1) #,'Holiday_count_x','Week_Year_x',	'Iso Year_y',	'Iso Weeknum_y',	'Holiday_count_y',	'Week_Year_y',	'Iso Year',	'Iso Weeknum',	'Holiday_count',	'Week_Year'
df2['IS_TET'] = df2['IS_TET'].fillna(0)


In [ ]:
###############################
#seasonality + date variables


def is_week(today, week_num):
    this_year, this_month = today.year, today.month
    weeks = calendar.monthcalendar(this_year, this_month)
    return 1 if today.day in weeks[week_num-1] else 0


#holiday check
def get_holiday_weeks(start_date, end_date, country):
    holidays_lis = holidays.CountryHoliday(country)
    holiday_dates = [date for date in pd.date_range(start_date, end_date) if date in holidays_lis]
    return list(set(str(date.isocalendar().week)+'-'+str(date.isocalendar().year) for date in holiday_dates))
    
# Calculate week flag
df2['IS_2ND_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: is_week(d, 2))
df2['IS_3RD_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: is_week(d, 3))
df2['IS_4TH_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: is_week(d, 4))

# Calculate holiday flags
holiday_weeks = get_holiday_weeks('2020-01-01', '2026-01-01', 'VN')
df2['IS_HOLIDAY_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: int(str(d.isocalendar().week)+'-'+str(d.isocalendar().year) in holiday_weeks))
df2['IS_4W_BF_HOLIDAY'] = df2['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=4)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=4)).isocalendar().year) in holiday_weeks))
df2['IS_3W_BF_HOLIDAY'] = df2['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=3)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=3)).isocalendar().year) in holiday_weeks))
df2['IS_2W_BF_HOLIDAY'] = df2['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=2)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=2)).isocalendar().year) in holiday_weeks))
df2['IS_1W_BF_HOLIDAY'] = df2['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=2)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=1)).isocalendar().year) in holiday_weeks))

In [ ]:

#category variables
df2['SUB_DIVISION_DESC'] = pd.factorize(df2['SUB_DIVISION_DESC'])[0]
df2['CATEGORY_DESC'] = pd.factorize(df2['CATEGORY_DESC'])[0]
df2['SUB_CATEGORY_DESC'] = pd.factorize(df2['SUB_CATEGORY_DESC'])[0]

2. Model training

In [ ]:
###################################################################3
### training and test

data_mdl = df2.copy()

sellout_w = 13
sellin_w = 13
target_columns = [f'SO_{horizon}W' for horizon in range(1, sellout_w+1)]+\
                    [f'SI_{horizon}W' for horizon in range(1, sellin_w+1)]


In [ ]:

###################################
#Pipeline
# Select Date as index

class Make_Date_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, date_column):
        self.date_column = date_column    
    
    def fit(self, X, y = None):
        return self    
    
    def transform(self, X, y = None):
        X_transformed = X.copy()
        X_transformed = X_transformed.set_index(self.date_column)
        X_transformed.index = pd.to_datetime(X_transformed.index)
        return X_transformed  
    
def model_evaluation(model, Xtrain, ytrain, Xtest, ytest):
    def fit_scatter_plot(X, y, set_name):
        y_fitted_values = model.predict(X)
        xmin = y.min()
        xmax = y.max()
        plt.scatter(x = y_fitted_values, y = y, alpha=0.25)
        x_line = np.linspace(xmin, xmax, 10)
        y_line = x_line
        plt.plot(x_line, y_line, 'r--')
        plt.axhline(0, color="black", linestyle="--")
        plt.xlabel('Prediction')
        plt.ylabel('True Value')
        plt.title(f'Plot of predicted values versus true values - {set_name} set')
    
    def plot_of_residuals(X, y, set_name):
        errors = model.predict(X) - np.reshape(np.array(y), (-1))
        plt.scatter(x = y, y = errors, alpha=0.25)
        plt.axhline(0, color="r", linestyle="--")
        plt.xlabel('True Value')
        plt.ylabel('Residual')
        plt.title(f'Plot of residuals - {set_name} set')
        
    def hist_of_residuals(X, y, set_name):
        errors = model.predict(X) - np.reshape(np.array(y), (-1))
        plt.hist(errors, bins = 100)
        plt.axvline(errors.mean(), color='k', linestyle='dashed', linewidth=1)
        plt.title(f'Histogram of residuals - {set_name} set')
    
    def DPA(y_true, y_pred):
        dpa = 100 - (((np.sum(np.abs(y_pred - y_true)))/(np.sum(y_true)))*100)
        return dpa

    def BIAS(y_true, y_pred):
        bias = (((np.sum(y_pred - y_true))/(np.sum(y_true)))*100)
        return bias
    
    fig = plt.figure(figsize = (16, 10))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    ax = fig.add_subplot(3, 2, 1)
    fit_scatter_plot(X = Xtrain, y = ytrain, set_name = 'train')
    
    ax = fig.add_subplot(3, 2, 2)
    fit_scatter_plot(X = Xtest, y = ytest, set_name = 'test')
    
    ax = fig.add_subplot(3, 2, 3)
    plot_of_residuals(X = Xtrain, y = ytrain, set_name = 'train')
    
    ax = fig.add_subplot(3, 2, 4)
    plot_of_residuals(X = Xtest, y = ytest, set_name = 'test')    

    ax = fig.add_subplot(3, 2, 5)
    hist_of_residuals(X = Xtrain, y = ytrain, set_name = 'train')   

    ax = fig.add_subplot(3, 2, 6)
    hist_of_residuals(X = Xtest, y = ytest, set_name = 'test')
    
    plt.show()
    
    y_pred_train = model.predict(Xtrain)
    print(f'RMSE train: {sqrt(mean_squared_error(ytrain, y_pred_train))}')
    print(f'DPA  train: {DPA(ytrain, y_pred_train)}')
    print(f'BIAS train: {BIAS(ytrain, y_pred_train)}')
    print()
    y_pred_test = model.predict(Xtest)
    print(f'RMSE test:  {sqrt(mean_squared_error(ytest, y_pred_test))}')
    print(f'DPA  test: {DPA(ytest, y_pred_test)}')
    print(f'BIAS test: {BIAS(ytest, y_pred_test)}')
    
    # Feature Importance
    importance = model._final_estimator.feature_importances_
    df_feature_importance=importance.argsort()
    df_feature_importance=pd.DataFrame({
        'column':Xtrain.columns[df_feature_importance],
        'importance':importance[df_feature_importance]
    })
    df_feature_importance = df_feature_importance[
                                            df_feature_importance['importance']>=0.01].copy().reset_index(drop=True)
    plt.figure(figsize=(16, 4))
    plt.barh(df_feature_importance['column'][-10:], df_feature_importance['importance'][-10:])
    plt.tick_params(axis='both', labelsize=10)
    plt.title('Model Feature Importance', size=20)
    plt.xlabel(' ', size=15)
    plt.tight_layout()
    plt.show()


2.1 10 highest SI SKUs

In [ ]:
# store variables
SO_test_result = pd.DataFrame()
SO_MAPE = []

# SI_test_result = pd.DataFrame()
# SI_MAPE = []

In [ ]:
first_test_date = dt.datetime.strptime('2023-03-01', '%Y-%m-%d')
end_test_date = dt.datetime.strptime('2023-06-30', '%Y-%m-%d')

for i in biggest_SKU.iloc[[0,1,2,4,5,6,7,8,9],0]: #
    for j in data_mdl['PROVINCE_REGION'].unique():
        print(f'STANDARD_SKU_DESC = {i} and PROVINCE_REGION = {j}')
        data_set = data_mdl[(data_mdl['STANDARD_SKU_DESC'] == i) & (data_mdl['PROVINCE_REGION'] == j)
                           & (data_mdl['YEAR'] >= 2022)] # 

        # Select Date as index

        data_set = data_set.set_index(data_set['PERIOD_TAG']).dropna()
        data_set.index = pd.to_datetime(data_set.index)

        historical_vars_window_5 = []
        for lag in range(1, 5):

            # data_set[f'SI_LAG_{lag}'] = data_set['WEEKLY_SI_ORDER'].shift(lag)
            data_set[f'SO_LAG_{lag}'] = data_set['WEEKLY_SO_ORDER'].shift(lag)
            historical_vars_window_5.append(f'SO_LAG_{lag}')
            if lag >= 2:
                # data_set[f'SI_WINDOW_STD_{lag}'] = data_set['WEEKLY_SI_ORDER'].rolling(window=lag).std()
                data_set[f'SO_WINDOW_STD_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).std()
                historical_vars_window_5.append(f'SO_WINDOW_STD_{lag}')

                # data_set[f'SI_WINDOW_MAX_{lag}'] = data_set['WEEKLY_SI_ORDER'].rolling(window=lag).max()
                data_set[f'SO_WINDOW_MAX_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).max()
                historical_vars_window_5.append(f'SO_WINDOW_MAX_{lag}')

                # data_set[f'SI_WINDOW_MIN_{lag}'] = data_set['WEEKLY_SI_ORDER'].rolling(window=lag).min()
                data_set[f'SO_WINDOW_MIN_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).min()
                historical_vars_window_5.append(f'SO_WINDOW_MIN_{lag}')

        # data_set['SI_CUMSUM_BY_WEEK'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SI_ORDER'].cumsum()
        data_set['SO_CUMSUM_BY_WEEK'] = data_set.groupby(['YEAR', 'MONTH'])['WEEKLY_SO_ORDER'].cumsum()

        # data_set['SI_SUM_BY_MONTH'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SI_ORDER'].transform(np.sum)
        data_set['SO_SUM_BY_MONTH'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SO_ORDER'].transform(np.sum)

        # data_set['SI_MEAN_BY_MONTH'] = data_set.groupby(['MONTH'])['WEEKLY_SI_ORDER'].transform(lambda x: x.sum()/12)
        data_set['SO_MEAN_BY_MONTH'] = data_set.groupby(['MONTH'])['WEEKLY_SO_ORDER'].transform(lambda x: x.sum()/12)

        # data_set['SI_MEAN_WINDOW_5'] = data_set['WEEKLY_SI_ORDER'].rolling(window=5).mean()
        data_set['SO_MEAN_WINDOW_5'] = data_set['WEEKLY_SO_ORDER'].rolling(window=5).mean()

        # data_set['SI_MEAN_WINDOW_10'] = data_set['WEEKLY_SI_ORDER'].rolling(window=10).mean()
        data_set['SO_MEAN_WINDOW_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).mean()

        data_set[f'SO_WINDOW_STD_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).std()

        data_set[f'SO_WINDOW_MAX_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).max()

        data_set[f'SO_WINDOW_MIN_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).min()

        data_set[f'SO_WINDOW_STD_13'] = data_set['WEEKLY_SO_ORDER'].rolling(window=13).std()

        data_set[f'SO_WINDOW_MAX_13'] = data_set['WEEKLY_SO_ORDER'].rolling(window=13).max()

        data_set[f'SO_WINDOW_MIN_13'] = data_set['WEEKLY_SO_ORDER'].rolling(window=13).min()

        data_set['SO_DIFF_1'] = data_set['WEEKLY_SO_ORDER'].diff()
        data_set['SO_DIFF_2'] = data_set['WEEKLY_SO_ORDER'].diff(periods=2)
        data_set['SO_DIFF_3'] = data_set['WEEKLY_SO_ORDER'].diff(periods=3)
        data_set['SO_DIFF_4'] = data_set['WEEKLY_SO_ORDER'].diff(periods=4)

        data_set['SO_MOVING_AVG_WINDOW_4']= (data_set['WEEKLY_SO_ORDER'].rolling(window = 4).sum())/4
        data_set['SO_MOVING_AVG_WINDOW_13']= (data_set['WEEKLY_SO_ORDER'].rolling(window = 13).sum())/13

        data_set['SO_RESIDUAL_WINDOW_4']= data_set['WEEKLY_SO_ORDER'] - data_set['SO_MOVING_AVG_WINDOW_4']
        data_set['SO_RESIDUAL_WINDOW_13']= data_set['WEEKLY_SO_ORDER'] - data_set['SO_MOVING_AVG_WINDOW_13']

        data_set['INVENTORY_BEGIN_WEEK'] = data_set['LAST_WEEK_INVENTORY'].shift(1)

        #historical vars within window of 5w 
        historical_vars_window_5.extend(['SO_DIFF_1', 'SO_DIFF_2', 'SO_DIFF_3', 'SO_DIFF_4',  'SO_CUMSUM_BY_WEEK', 'SO_SUM_BY_MONTH', 'SO_MOVING_AVG_WINDOW_4', 'SO_RESIDUAL_WINDOW_4'])
        
        
        #IS_1ST_WEEK	IS_2ND_WEEK	IS_3RD_WEEK	IS_4TH_WEEK	IS_HOLIDAY_WEEK	IS_4W_BF_HOLIDAY	IS_3W_BF_HOLIDAY	IS_2W_BF_HOLIDAY

        analysis = data_set['WEEKLY_SO_ORDER'].copy()

        decompose_result_add = seasonal_decompose(analysis, model='additive')
            
        data_set['SO_SEASONAL'] = decompose_result_add.seasonal
        data_set = data_set.drop(['SUB_DIVISION_DESC',	'CATEGORY_DESC',	'SUB_CATEGORY_DESC'], axis= 1)

        for horizon_prediction in target_columns[0:13]:
            start_time = time.time()
            
            print(f'|---------------------------\t{horizon_prediction}\t---------------------------|')
            if horizon_prediction[:2] == 'SO':
                nr_horizon = int(horizon_prediction[3:-1])
                type_prediction = 'Sellout'
                target_name = 'Sellout'

                #SO training
            print('Creating a SellOut training set')
            all_set = data_set.copy()

            all_set[f'SO_SEASONAL_{nr_horizon}W'] =  all_set['SO_SEASONAL'].shift(-nr_horizon)

            all_set[f'WEEK_NUM_{nr_horizon}W'] =  all_set['WEEK_NUM'].shift(-nr_horizon)
            all_set[f'HOLIDAY_COUNT_{nr_horizon}W'] =  all_set['HOLIDAY_COUNT'].shift(-nr_horizon)
            all_set[f'IS_HOLIDAY_WEEK_{nr_horizon}W'] =  all_set['IS_HOLIDAY_WEEK'].shift(-nr_horizon)
            all_set[f'IS_4W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_4W_BF_HOLIDAY'].shift(-nr_horizon)
            all_set[f'IS_3W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_3W_BF_HOLIDAY'].shift(-nr_horizon)
            all_set[f'IS_2W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_2W_BF_HOLIDAY'].shift(-nr_horizon)
            all_set[f'IS_1W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_1W_BF_HOLIDAY'].shift(-nr_horizon)

            all_set[horizon_prediction] = all_set['WEEKLY_SO_ORDER'].shift(-nr_horizon).copy()

            all_set = all_set.dropna()
            if nr_horizon <= 4:

                SO_train_set = all_set.loc[(all_set['PERIOD_TAG'] < first_test_date), :]
                X_train = SO_train_set.drop(list(all_set.filter(regex='SI_'))+[horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                y_train = SO_train_set[horizon_prediction].copy()
                print(X_train.columns)

                print('--------------------------------------------------------')

                ##### Sell Out Test ##

                SO_test_set = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                            (all_set['PERIOD_TAG'] <= end_test_date ), :]
                X_test =  SO_test_set.drop(list(all_set.filter(regex='SI_'))+[horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                y_test =  SO_test_set[horizon_prediction].copy()
            else:
                
                SO_train_set = all_set.loc[(all_set['PERIOD_TAG'] < first_test_date), :]
                X_train = SO_train_set.drop(list(all_set.filter(regex='SI_'))+ historical_vars_window_5 + [horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                y_train = SO_train_set[horizon_prediction].copy()
                print(X_train.columns)

                print('--------------------------------------------------------')

                ##### Sell Out Test ##

                SO_test_set = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                            (all_set['PERIOD_TAG'] <= end_test_date ), :]
                X_test =  SO_test_set.drop(list(all_set.filter(regex='SI_')) + historical_vars_window_5 + [horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                y_test =  SO_test_set[horizon_prediction].copy()


            ################################################################ Add a forecast to a dataset ####################
            random_forest_pipeline = Pipeline(steps=[('regressor', RandomForestRegressor(random_state=123, n_jobs=3))])

            param_random_forest = {
                'regressor__max_depth': [5, 10, 15],
                'regressor__min_samples_leaf': [3, 7],
                'regressor__n_estimators': [100],
                'regressor__criterion': ['absolute_error'],
            }


            if nr_horizon in [1, 2, 3, 4]:
                random_forest = GridSearchCV(random_forest_pipeline, param_random_forest, 
                                            cv=TimeSeriesSplit(n_splits=5).split(X_train), n_jobs=3, 
                                            scoring = 'neg_mean_absolute_error', verbose=10, error_score = 'raise')
                random_forest.fit(X_train, y_train)
                # print("The best estimator: ", random_forest.best_estimator_,
                #     "\nThe best score: ", random_forest.best_score_,
                #     "\nThe best parameters: ", random_forest.best_params_, sep="\n")

                random_forest_model = random_forest.best_estimator_
                best_params = random_forest.best_params_
            else:
                random_forest_model = random_forest_pipeline.set_params(**best_params)
                random_forest_model.fit(X_train, y_train)
            

            SO_test_set[f'Pred_RF_{horizon_prediction}W'] = random_forest_model.predict(X_test)

            if nr_horizon in [1, 2, 3, 5, 6, 10, 13]:
                print('\nEvaluation charts')
                ## Evaluation check#########################################
                model_evaluation(model = random_forest_model,
                                Xtrain = X_train, ytrain = y_train,
                                Xtest  = X_test,  ytest  = y_test)

            RF_SO_MAPE = 1/(len(y_test))*sum(abs(y_test-SO_test_set[f'Pred_RF_{horizon_prediction}W'])/y_test)
            
            print(f"Forecast Accuracy RF MAPE: {mean_absolute_percentage_error(y_test, SO_test_set[f'Pred_RF_{horizon_prediction}W'])}")


            plt.figure(figsize=(25,6))
            plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
            plt.plot(range(0,len(y_test)), SO_test_set[f'Pred_RF_{horizon_prediction}W'], color = 'blue', linewidth=0.8)
            plt.legend(['Actual SO','Predicted SO' ])
            plt.xlabel('Timestamp')
            plt.title(f"{i} - {j} - Actual vs prediction")
            plt.show()
        


        #################################### XGB Model###############################################################################

            # matrix_train = xgb.DMatrix(X_train, label = y_train)
            # matrix_test = xgb.DMatrix(X_test, label = y_test)

            # # Run XGB 
            # XGBoost
#             cv_split = TimeSeriesSplit(n_splits=3, test_size=None)
#             model = XGBRegressor()
#             parameters = {
#                 "max_depth": [3, 4, 6, 5, 10],
#                 "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
#                 "n_estimators": [100, 300, 500, 700, 900, 1000],
#                 "colsample_bytree": [0.3, 0.5, 0.7],
#                 "early_stopping_rounds": [20]
#             }

#             XGB = GridSearchCV(estimator=model, cv=cv_split, param_grid=parameters)
#             XGB.fit(X_train, y_train)
                
#             XGB_model = XGB.best_estimator_
#             XGB_best_params = XGB.best_params_
            
            
            # xgb_model = xgb.train(params={'objective':'reg:linear','eval_metric':'mape'}
            #                 , dtrain = matrix_train, num_boost_round = 500, 
            #                 early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

#             SO_test_set[f'Pred_XGB_{horizon_prediction}W'] = XGB.predict(X_test)

            # XGB_SO_MAPE = 1/(len(y_test))*sum(abs((y_test-SO_test_set[f'Pred_XGB_{horizon_prediction}'])/y_test))
            
            # print(f'Forecast Accuracy XGB MAPE {XGB_SO_MAPE}')
            
            #SO_MAPE.append([i,j,RF_SO_MAPE]) #,XGB_SO_MAPE

#             plt.figure(figsize=(25,6))
#             plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
#             plt.plot(range(0,len(y_test)), SO_test_set[f'Pred_XGB_{horizon_prediction}W'], color = 'blue', linewidth=0.8)
#             plt.legend(['Actual SO','Predicted SO' ])
#             plt.xlabel('Timestamp')
#             plt.title("Actual vs prediction")
#             plt.show()

#             SO_test_result = pd.concat([SO_test_result, SO_test_set])
            
#             elapsed_time = time.time() - start_time
#             print("Finished iterating in: " + str(int(elapsed_time))  + " seconds")
#             print('\n')


        # #####################################################SI training###################################################################
        # print('Creating a Sell In training set')
        # SI_train_set = data_set.loc[data_set['PERIOD_TAG'] < first_test_date, :]
        # X_train = SI_train_set.drop(list(data_set.filter(regex='^SO'))+['WEEKLY_SI_ORDER','PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION'], axis = 1).copy()
        # y_train = SI_train_set['WEEKLY_SI_ORDER'].copy()

        # print('--------------------------------------------------------')

        # #####%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Sell IN Test %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%#################################
        # print('Creating a Sell In test set')

        # SI_test_set = data_set.loc[(data_set['PERIOD_TAG'] >= first_test_date) & (data_set['PERIOD_TAG'] <= end_test_date), :]

        # X_test =  SI_test_set.drop(list(data_set.filter(regex='^SO'))+['WEEKLY_SI_ORDER','PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION'], axis = 1).copy()
        # y_test =  SI_test_set['WEEKLY_SI_ORDER'].copy()

        # print('--------------------------------------------------------')

        #  ################################################################ Add a forecast to a dataset ####################
        # random_forest_pipeline = Pipeline(steps=[('regressor', RandomForestRegressor(random_state=123, n_jobs=3))])

        # param_random_forest = {
        #     'regressor__max_depth': [5, 10, 15],
        #     'regressor__min_samples_leaf': [3, 7],
        #     'regressor__n_estimators': [100],
        #     'regressor__criterion': ['absolute_error'],
        # }

        # random_forest = GridSearchCV(random_forest_pipeline, param_random_forest, 
        #                                  cv=TimeSeriesSplit(n_splits=5).split(X_train), n_jobs=3, 
        #                                  scoring = 'neg_mean_absolute_error', verbose=10, error_score = 'raise')
        # random_forest.fit(X_train, y_train)
        # print("The best estimator: ", random_forest.best_estimator_,
        #       "\nThe best score: ", random_forest.best_score_,
        #       "\nThe best parameters: ", random_forest.best_params_, sep="\n")

        # random_forest_model = random_forest.best_estimator_
        # best_params = random_forest.best_params_
        
        # print('\nEvaluation charts')

        # ### Evaluation check#########################################
        # model_evaluation(model = random_forest_model,
        #                  Xtrain = X_train, ytrain = y_train,
        #                  Xtest  = X_test,  ytest  = y_test)

        # SI_test_set['Pred_RF_SI'] = random_forest_model.predict(X_test)

        # RF_SI_MAPE = 1/(len(y_test))*sum(abs(y_test-SI_test_set['Pred_RF_SI'])/y_test)

        # print(f'Sell In Forecast Accuracy RF MAPE {RF_SI_MAPE}')
        
        

        # plt.figure(figsize=(25,6))
        # plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
        # plt.plot(range(0,len(y_test)), SI_test_set['Pred_RF_SI'], color = 'blue', linewidth=0.8)
        # plt.legend(['Actual SI','Predicted SI' ])
        # plt.xlabel('Timestamp')
        # plt.title("Actual vs prediction")
        # plt.show()
        
        
        # #################################### XGB Model###############################################################################

        # matrix_train = xgb.DMatrix(X_train, label = y_train)
        # matrix_test = xgb.DMatrix(X_test, label = y_test)
        # # Run XGB 
        # xgb_model = xgb.train(params={'objective':'reg:linear','eval_metric':'mape'}
        #                 , dtrain = matrix_train, num_boost_round = 500, 
        #                 early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

        # SI_test_set['Pred_XGB_SI'] = xgb_model.predict(matrix_test)

        # XGB_SI_MAPE = 1/(len(y_test))*sum(abs((y_test-SI_test_set['Pred_XGB_SI'])/y_test))

        # SI_test_pre = SI_test_set.copy()
        
        # SI_test_result = pd.concat([ SI_test_result, SI_test_pre])
        
        # SI_MAPE.append([i,j,RF_SI_MAPE,XGB_SI_MAPE])


2.2 2 Others SKUs in top 100 highest SI

In [ ]:
# store variables
SO_test_result_top_100 = pd.DataFrame()
# SO_MAPE_top_100 = []

# SI_test_result = pd.DataFrame()
# SI_MAPE_top_100 = []

In [ ]:
first_test_date = dt.datetime.strptime('2023-01-01', '%Y-%m-%d')
end_test_date = dt.datetime.strptime('2023-03-31', '%Y-%m-%d')

for i in biggest_SKU.iloc[10:100,0]:
    for j in data_mdl['PROVINCE_REGION'].unique():
        print(f'STANDARD_SKU_DESC = {i} and PROVINCE_REGION = {j}')
        data_set = data_mdl[(data_mdl['STANDARD_SKU_DESC'] == i) & (data_mdl['PROVINCE_REGION'] == j)
                           & (data_mdl['YEAR'] >= 2022)] # 
        if sum(data_set['WEEKLY_SO_ORDER'] > 0):

            # Select Date as index

            data_set = data_set.set_index(data_set['PERIOD_TAG']).dropna()
            data_set.index = pd.to_datetime(data_set.index)

            historical_vars_window_5 = []
            for lag in range(1, 5):

                # data_set[f'SI_LAG_{lag}'] = data_set['WEEKLY_SI_ORDER'].shift(lag)
                data_set[f'SO_LAG_{lag}'] = data_set['WEEKLY_SO_ORDER'].shift(lag)
                historical_vars_window_5.append(f'SO_LAG_{lag}')
                if lag >= 2:
                    # data_set[f'SI_WINDOW_STD_{lag}'] = data_set['WEEKLY_SI_ORDER'].rolling(window=lag).std()
                    data_set[f'SO_WINDOW_STD_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).std()
                    historical_vars_window_5.append(f'SO_WINDOW_STD_{lag}')

                    # data_set[f'SI_WINDOW_MAX_{lag}'] = data_set['WEEKLY_SI_ORDER'].rolling(window=lag).max()
                    data_set[f'SO_WINDOW_MAX_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).max()
                    historical_vars_window_5.append(f'SO_WINDOW_MAX_{lag}')

                    # data_set[f'SI_WINDOW_MIN_{lag}'] = data_set['WEEKLY_SI_ORDER'].rolling(window=lag).min()
                    data_set[f'SO_WINDOW_MIN_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).min()
                    historical_vars_window_5.append(f'SO_WINDOW_MIN_{lag}')

            # data_set['SI_CUMSUM_BY_WEEK'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SI_ORDER'].cumsum()
            data_set['SO_CUMSUM_BY_WEEK'] = data_set.groupby(['YEAR', 'MONTH'])['WEEKLY_SO_ORDER'].cumsum()

            # data_set['SI_SUM_BY_MONTH'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SI_ORDER'].transform(np.sum)
            data_set['SO_SUM_BY_MONTH'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SO_ORDER'].transform(np.sum)

            # data_set['SI_MEAN_BY_MONTH'] = data_set.groupby(['MONTH'])['WEEKLY_SI_ORDER'].transform(lambda x: x.sum()/12)
            data_set['SO_MEAN_BY_MONTH'] = data_set.groupby(['MONTH'])['WEEKLY_SO_ORDER'].transform(lambda x: x.sum()/12)

            # data_set['SI_MEAN_WINDOW_5'] = data_set['WEEKLY_SI_ORDER'].rolling(window=5).mean()
            data_set['SO_MEAN_WINDOW_5'] = data_set['WEEKLY_SO_ORDER'].rolling(window=5).mean()

            # data_set['SI_MEAN_WINDOW_10'] = data_set['WEEKLY_SI_ORDER'].rolling(window=10).mean()
            data_set['SO_MEAN_WINDOW_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).mean()

            data_set[f'SO_WINDOW_STD_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).std()

            data_set[f'SO_WINDOW_MAX_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).max()

            data_set[f'SO_WINDOW_MIN_10'] = data_set['WEEKLY_SO_ORDER'].rolling(window=10).min()

            data_set[f'SO_WINDOW_STD_13'] = data_set['WEEKLY_SO_ORDER'].rolling(window=13).std()

            data_set[f'SO_WINDOW_MAX_13'] = data_set['WEEKLY_SO_ORDER'].rolling(window=13).max()

            data_set[f'SO_WINDOW_MIN_13'] = data_set['WEEKLY_SO_ORDER'].rolling(window=13).min()

            data_set['SO_DIFF_1'] = data_set['WEEKLY_SO_ORDER'].diff()
            data_set['SO_DIFF_2'] = data_set['WEEKLY_SO_ORDER'].diff(periods=2)
            data_set['SO_DIFF_3'] = data_set['WEEKLY_SO_ORDER'].diff(periods=3)
            data_set['SO_DIFF_4'] = data_set['WEEKLY_SO_ORDER'].diff(periods=4)

            data_set['SO_MOVING_AVG_WINDOW_4']= (data_set['WEEKLY_SO_ORDER'].rolling(window = 4).sum())/4
            data_set['SO_MOVING_AVG_WINDOW_13']= (data_set['WEEKLY_SO_ORDER'].rolling(window = 13).sum())/13

            data_set['SO_RESIDUAL_WINDOW_4']= data_set['WEEKLY_SO_ORDER'] - data_set['SO_MOVING_AVG_WINDOW_4']
            data_set['SO_RESIDUAL_WINDOW_13']= data_set['WEEKLY_SO_ORDER'] - data_set['SO_MOVING_AVG_WINDOW_13']

            data_set['INVENTORY_BEGIN_WEEK'] = data_set['LAST_WEEK_INVENTORY'].shift(1)

            #historical vars within window of 5w 
            historical_vars_window_5.extend(['SO_DIFF_1', 'SO_DIFF_2', 'SO_DIFF_3', 'SO_DIFF_4',  'SO_CUMSUM_BY_WEEK', 'SO_SUM_BY_MONTH', 'SO_MOVING_AVG_WINDOW_4', 'SO_RESIDUAL_WINDOW_4'])


            #IS_1ST_WEEK	IS_2ND_WEEK	IS_3RD_WEEK	IS_4TH_WEEK	IS_HOLIDAY_WEEK	IS_4W_BF_HOLIDAY	IS_3W_BF_HOLIDAY	IS_2W_BF_HOLIDAY

            analysis = data_set['WEEKLY_SO_ORDER'].copy()

            decompose_result_add = seasonal_decompose(analysis, model='additive')

            data_set['SO_SEASONAL'] = decompose_result_add.seasonal
            data_set = data_set.drop(['SUB_DIVISION_DESC',	'CATEGORY_DESC',	'SUB_CATEGORY_DESC'], axis= 1)

            for horizon_prediction in target_columns[0:5]:
                start_time = time.time()
                print(f'|---------------------------\t{horizon_prediction}\t---------------------------|')
                if horizon_prediction[:2] == 'SO':
                    nr_horizon = int(horizon_prediction[3:-1])
                    type_prediction = 'Sellout'
                    target_name = 'Sellout'

                    #SO training
                print('Creating a SellOut training set')
                all_set = data_set.copy()

                all_set[f'SO_SEASONAL_{nr_horizon}W'] =  all_set['SO_SEASONAL'].shift(-nr_horizon)

                all_set[f'WEEK_NUM_{nr_horizon}W'] =  all_set['WEEK_NUM'].shift(-nr_horizon)
                all_set[f'HOLIDAY_COUNT_{nr_horizon}W'] =  all_set['HOLIDAY_COUNT'].shift(-nr_horizon)
                all_set[f'IS_HOLIDAY_WEEK_{nr_horizon}W'] =  all_set['IS_HOLIDAY_WEEK'].shift(-nr_horizon)
                all_set[f'IS_4W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_4W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_3W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_3W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_2W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_2W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_1W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_1W_BF_HOLIDAY'].shift(-nr_horizon)

                all_set[horizon_prediction] = all_set['WEEKLY_SO_ORDER'].shift(-nr_horizon).copy()

                all_set = all_set.dropna()
                if nr_horizon <= 4:

                    SO_train_set = all_set.loc[(all_set['PERIOD_TAG'] < first_test_date), :]
                    X_train = SO_train_set.drop(list(all_set.filter(regex='SI_'))+[horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                    y_train = SO_train_set[horizon_prediction].copy()
                    print(X_train.columns)

                    print('--------------------------------------------------------')

                    ##### Sell Out Test ##

                    SO_test_set = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                                (all_set['PERIOD_TAG'] <= end_test_date ), :]
                    X_test =  SO_test_set.drop(list(all_set.filter(regex='SI_'))+[horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                    y_test =  SO_test_set[horizon_prediction].copy()
                else:

                    SO_train_set = all_set.loc[(all_set['PERIOD_TAG'] < first_test_date), :]
                    X_train = SO_train_set.drop(list(all_set.filter(regex='SI_'))+ historical_vars_window_5 + [horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                    y_train = SO_train_set[horizon_prediction].copy()
                    print(X_train.columns)

                    print('--------------------------------------------------------')

                    ##### Sell Out Test ##

                    SO_test_set = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                                (all_set['PERIOD_TAG'] <= end_test_date ), :]
                    X_test =  SO_test_set.drop(list(all_set.filter(regex='SI_')) + historical_vars_window_5 + [horizon_prediction]+['PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR','MONTH','LAST_WEEK_INVENTORY'], axis = 1).copy()
                    y_test =  SO_test_set[horizon_prediction].copy()

                print('--------------------------------------------------------')

                ################################################################ Add a forecast to a dataset ####################
                # random_forest_pipeline = Pipeline(steps=[('regressor', RandomForestRegressor(random_state=123, n_jobs=3))])

                # param_random_forest = {
                #     'regressor__max_depth': [5, 10, 15],
                #     'regressor__min_samples_leaf': [3, 7],
                #     'regressor__n_estimators': [100],
                #     'regressor__criterion': ['absolute_error'],
                # }


                # # print("The best estimator: ", random_forest.best_estimator_,
                # #     "\nThe best score: ", random_forest.best_score_,
                # #     "\nThe best parameters: ", random_forest.best_params_, sep="\n")

                # random_forest_model = random_forest.best_estimator_
                # best_params = random_forest.best_params_

                if nr_horizon in [1, 2, 3, 4, 5]:
                    random_forest = GridSearchCV(random_forest_pipeline, param_random_forest, 
                                                cv=TimeSeriesSplit(n_splits=5).split(X_train), n_jobs=3, 
                                                scoring = 'neg_mean_absolute_error', verbose=10, error_score = 'raise')
                    random_forest.fit(X_train, y_train)
                    # print("The best estimator: ", random_forest.best_estimator_,
                    #     "\nThe best score: ", random_forest.best_score_,
                    #     "\nThe best parameters: ", random_forest.best_params_, sep="\n")

                    random_forest_model = random_forest.best_estimator_
                    best_params = random_forest.best_params_
                else:
                    random_forest_model = random_forest_pipeline.set_params(**best_params)
                    random_forest_model.fit(X_train, y_train)


                SO_test_set[f'Pred_RF_{horizon_prediction}W'] = random_forest_model.predict(X_test)

                if nr_horizon in [1, 2, 3, 6]:
                    print('\nEvaluation charts')
                    ## Evaluation check#########################################
                    model_evaluation(model = random_forest_model,
                                    Xtrain = X_train, ytrain = y_train,
                                    Xtest  = X_test,  ytest  = y_test)

                RF_SO_MAPE = 1/(len(y_test))*sum(abs(y_test-SO_test_set[f'Pred_RF_{horizon_prediction}W'])/y_test)

                print(f'Forecast Accuracy RF MAPE {RF_SO_MAPE}')

                plt.figure(figsize=(25,6))
                plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
                plt.plot(range(0,len(y_test)), SO_test_set[f'Pred_RF_{horizon_prediction}W'], color = 'blue', linewidth=0.8)
                plt.legend(['Actual SO','Predicted SO' ])
                plt.xlabel('Timestamp')
                plt.title(f"{i} - {j} - Actual vs prediction")
                plt.show()


                elapsed_time = time.time() - start_time
                print("Finished iterating in: " + str(int(elapsed_time))  + " seconds")
                print('\n')
            #################################### XGB Model###############################################################################

                # matrix_train = xgb.DMatrix(X_train, label = y_train)
                # matrix_test = xgb.DMatrix(X_test, label = y_test)

                # # Run XGB 
                # xgb_model = xgb.train(params={'objective':'reg:linear','eval_metric':'mape'}
                #                 , dtrain = matrix_train, num_boost_round = 500, 
                #                 early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

                # SO_test_set[f'Pred_XGB_{horizon_prediction}'] = xgb_model.predict(matrix_test)

                # XGB_SO_MAPE = 1/(len(y_test))*sum(abs((y_test-SO_test_set[f'Pred_XGB_{horizon_prediction}'])/y_test))

                # print(f'Forecast Accuracy XGB MAPE {XGB_SO_MAPE}')

                #SO_MAPE.append([i,j,RF_SO_MAPE]) #,XGB_SO_MAPE

                # plt.figure(figsize=(25,6))
                # plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
                # plt.plot(range(0,len(y_test)), SO_test_set[f'Pred_XGB_{horizon_prediction}'], color = 'blue', linewidth=0.8)
                # plt.legend(['Actual SO','Predicted SO' ])
                # plt.xlabel('Timestamp')
                # plt.title("Actual vs prediction")
                # plt.show()

                SO_test_result_top_100 = pd.concat([SO_test_result_top_100, SO_test_set])





            # #####################################################SI training###################################################################
            # print('Creating a Sell In training set')
            # SI_train_set = data_set.loc[data_set['PERIOD_TAG'] < first_test_date, :]
            # X_train = SI_train_set.drop(list(data_set.filter(regex='^SO'))+['WEEKLY_SI_ORDER','PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION'], axis = 1).copy()
            # y_train = SI_train_set['WEEKLY_SI_ORDER'].copy()

            # print('--------------------------------------------------------')

            # #####%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Sell IN Test %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%#################################
            # print('Creating a Sell In test set')

            # SI_test_set = data_set.loc[(data_set['PERIOD_TAG'] >= first_test_date) & (data_set['PERIOD_TAG'] <= end_test_date), :]

            # X_test =  SI_test_set.drop(list(data_set.filter(regex='^SO'))+['WEEKLY_SI_ORDER','PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION'], axis = 1).copy()
            # y_test =  SI_test_set['WEEKLY_SI_ORDER'].copy()

            # print('--------------------------------------------------------')

            #  ################################################################ Add a forecast to a dataset ####################
            # random_forest_pipeline = Pipeline(steps=[('regressor', RandomForestRegressor(random_state=123, n_jobs=3))])

            # param_random_forest = {
            #     'regressor__max_depth': [5, 10, 15],
            #     'regressor__min_samples_leaf': [3, 7],
            #     'regressor__n_estimators': [100],
            #     'regressor__criterion': ['absolute_error'],
            # }

            # random_forest = GridSearchCV(random_forest_pipeline, param_random_forest, 
            #                                  cv=TimeSeriesSplit(n_splits=5).split(X_train), n_jobs=3, 
            #                                  scoring = 'neg_mean_absolute_error', verbose=10, error_score = 'raise')
            # random_forest.fit(X_train, y_train)
            # print("The best estimator: ", random_forest.best_estimator_,
            #       "\nThe best score: ", random_forest.best_score_,
            #       "\nThe best parameters: ", random_forest.best_params_, sep="\n")

            # random_forest_model = random_forest.best_estimator_
            # best_params = random_forest.best_params_

            # print('\nEvaluation charts')

            # ### Evaluation check#########################################
            # model_evaluation(model = random_forest_model,
            #                  Xtrain = X_train, ytrain = y_train,
            #                  Xtest  = X_test,  ytest  = y_test)

            # SI_test_set['Pred_RF_SI'] = random_forest_model.predict(X_test)

            # RF_SI_MAPE = 1/(len(y_test))*sum(abs(y_test-SI_test_set['Pred_RF_SI'])/y_test)

            # print(f'Sell In Forecast Accuracy RF MAPE {RF_SI_MAPE}')



            # plt.figure(figsize=(25,6))
            # plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
            # plt.plot(range(0,len(y_test)), SI_test_set['Pred_RF_SI'], color = 'blue', linewidth=0.8)
            # plt.legend(['Actual SI','Predicted SI' ])
            # plt.xlabel('Timestamp')
            # plt.title("Actual vs prediction")
            # plt.show()


            # #################################### XGB Model###############################################################################

            # matrix_train = xgb.DMatrix(X_train, label = y_train)
            # matrix_test = xgb.DMatrix(X_test, label = y_test)
            # # Run XGB 
            # xgb_model = xgb.train(params={'objective':'reg:linear','eval_metric':'mape'}
            #                 , dtrain = matrix_train, num_boost_round = 500, 
            #                 early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

            # SI_test_set['Pred_XGB_SI'] = xgb_model.predict(matrix_test)

            # XGB_SI_MAPE = 1/(len(y_test))*sum(abs((y_test-SI_test_set['Pred_XGB_SI'])/y_test))

            # SI_test_pre = SI_test_set.copy()

            # SI_test_result = pd.concat([ SI_test_result, SI_test_pre])

            # SI_MAPE.append([i,j,RF_SI_MAPE,XGB_SI_MAPE])
